In [1]:
import os
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import torchaudio.transforms
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torchvision.transforms import v2 as T
from torchvision import models
from tqdm.notebook import tqdm
import warnings

In [2]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

Using device: cuda


In [4]:
class TimeShift(torch.nn.Module):
    """ Dịch chuyển spectrogram theo trục thời gian một cách ngẫu nhiên. """
    def __init__(self, p=0.5, max_shift_percent=0.4):
        super().__init__()
        self.p = p
        self.max_shift_percent = max_shift_percent

    def forward(self, spec):
        if torch.rand(1) < self.p:
            time_steps = spec.shape[-1]
            shift_amount = int(torch.rand(1) * self.max_shift_percent * time_steps)
            if torch.rand(1) < 0.5:
                shift_amount = -shift_amount
            return torch.roll(spec, shifts=shift_amount, dims=-1)
        return spec

class SpecNoise(torch.nn.Module):
    """ Thêm nhiễu Gaussian trực tiếp vào spectrogram. """
    def __init__(self, p=0.5, noise_level=0.05):
        super().__init__()
        self.p = p
        self.noise_level = noise_level

    def forward(self, spec):
        if torch.rand(1) < self.p:
            noise = torch.randn_like(spec) * self.noise_level
            return spec + noise
        return spec

train_transform = T.Compose([
    torchaudio.transforms.TimeMasking(time_mask_param=80),
    torchaudio.transforms.FrequencyMasking(freq_mask_param=25),
    TimeShift(p=0.5, max_shift_percent=0.4),
    SpecNoise(p=0.3, noise_level=0.03),
])

val_transform = None
print("Đã định nghĩa xong pipeline Augmentation mới cho Spectrogram.")

Đã định nghĩa xong pipeline Augmentation mới cho Spectrogram.


In [5]:
def mixup_data(x, y, alpha=0.4, use_cuda=True):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [6]:
class MobileNetV3_AudioClassifier(nn.Module):
    def __init__(self, num_classes, input_channels=1, pretrained=True, pretrained_weights_path=None):
        super().__init__()
        weights = models.MobileNet_V3_Large_Weights.DEFAULT if pretrained else None
        self.mobilenet = models.mobilenet_v3_large(weights=weights)

        # Thay thế lớp Conv2d đầu tiên để phù hợp với số kênh đầu vào (1 cho spectrogram)
        original_conv = self.mobilenet.features[0][0]
        new_first_conv = nn.Conv2d(
            in_channels=input_channels,
            out_channels=original_conv.out_channels,
            kernel_size=original_conv.kernel_size,
            stride=original_conv.stride,
            padding=original_conv.padding,
            bias=original_conv.bias is not None
        )
        self.mobilenet.features[0][0] = new_first_conv

        # Thay thế lớp classifier
        num_ftrs = self.mobilenet.classifier[-1].in_features
        self.mobilenet.classifier[-1] = nn.Linear(num_ftrs, num_classes)

        if pretrained_weights_path:
            print(f"Loading pretrained weights from: {pretrained_weights_path}")
            checkpoint = torch.load(pretrained_weights_path, map_location=device, weights_only=True)
            state_dict = {k: v for k, v in checkpoint.items() if k in self.state_dict()}
            self.load_state_dict(state_dict, strict=False)

    def forward(self, x):
        if x.dim() == 3:
            x = x.unsqueeze(1)  # Thêm chiều kênh nếu cần

In [7]:
class SpectrogramDataset(Dataset):
    def __init__(self, metadata_df, data_dir, transform=None, target_tdim=216):
        super().__init__()
        self.metadata = metadata_df
        self.data_dir = data_dir
        self.transform = transform
        self.target_tdim = target_tdim
        self.class_names = sorted(self.metadata['label'].unique())
        self.class_to_id = {name: i for i, name in enumerate(self.class_names)}

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        file_path = os.path.join(self.data_dir, row['tensor_path'])
        label = self.class_to_id[row['label']]

        try:
            spectrogram = torch.load(file_path)
            current_tdim = spectrogram.shape[2]
            if current_tdim > self.target_tdim:
                spectrogram = spectrogram[:, :, :self.target_tdim]
            elif current_tdim < self.target_tdim:
                spectrogram = F.pad(spectrogram, (0, self.target_tdim - current_tdim))

            if self.transform:
                spectrogram = self.transform(spectrogram)
            return spectrogram, label
        except Exception as e:
            print(f"Error loading or processing file {file_path}: {e}")
            return None

def collate_fn_remove_none(batch):
    batch = [item for item in batch if item is not None]
    if not batch: return None, None
    samples, labels = zip(*batch)
    return torch.stack(samples), torch.tensor(labels)

In [8]:
def train_and_evaluate(model, train_loader, val_loader, epochs, checkpoint_path, metrics_csv_path):
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.5, patience=5, verbose=False)
    scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
    best_f1 = 0.0

    if not os.path.exists(metrics_csv_path):
        pd.DataFrame(columns=['Epoch', 'Accuracy', 'Precision', 'Recall', 'F1-Score']).to_csv(metrics_csv_path, index=False)

    for epoch in range(epochs):
        model.train()
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
        for inputs, labels in loop:
            if inputs is None: continue
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad(set_to_none=True)

            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        model.eval()
        val_preds, val_labels = [], []
        with torch.no_grad():
            for inputs, labels in val_loader:
                if inputs is None: continue
                inputs, labels = inputs.to(device), labels.to(device)
                with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                    outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                val_preds.extend(predicted.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        if len(val_labels) > 0:
            current_f1 = f1_score(val_labels, val_preds, average='weighted', zero_division=0)
            if current_f1 > best_f1:
                best_f1 = current_f1
                current_accuracy = accuracy_score(val_labels, val_preds)
                current_precision = precision_score(val_labels, val_preds, average='weighted', zero_division=0)
                current_recall = recall_score(val_labels, val_preds, average='weighted', zero_division=0)

                print(f"\nEpoch {epoch+1}: New best F1-score: {best_f1:.4f}")
                print(f"Accuracy: {current_accuracy:.4f}, Precision: {current_precision:.4f}, Recall: {current_recall:.4f}")
                print(f"Saving checkpoint to {checkpoint_path}")
                torch.save(model.state_dict(), checkpoint_path)

                metrics_df = pd.DataFrame([{'Epoch': epoch + 1, 'Accuracy': current_accuracy, 'Precision': current_precision, 'Recall': current_recall, 'F1-Score': current_f1}])
                metrics_df.to_csv(metrics_csv_path, mode='a', header=False, index=False)
            scheduler.step(current_f1)

    print(f"Finished Training for this run. Best F1-score: {best_f1:.4f}")
    return best_f1

In [9]:
ROOT_DIR = r'C:\Users\PC\lavar\audiotwin_checkpoints_medium'
METADATA_PATH = os.path.join(ROOT_DIR, 'spectrogram_tensor_metadata.csv')
CHECKPOINT_DIR = os.path.join(ROOT_DIR, 'training_checkpoints')
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

DATA_PERCENTAGES = [1.0]
results = {}

print("Preparing dataset...")
full_metadata = pd.read_csv(METADATA_PATH)
num_classes = len(full_metadata['label'].unique())
print(f"Number of classes: {num_classes}")

indices = np.arange(len(full_metadata))
labels = full_metadata['label'].values
train_indices, test_indices, _, _ = train_test_split(
    indices, labels, test_size=0.2, random_state=SEED, stratify=labels
)
print(f"Data split: {len(train_indices)} train, {len(test_indices)} test.")

test_metadata = full_metadata.iloc[test_indices]
test_dataset = SpectrogramDataset(metadata_df=test_metadata, data_dir=ROOT_DIR, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=32, num_workers=0, collate_fn=collate_fn_remove_none)

# 2. Cấu hình mô hình MobileNetV3
model_configs = {
    "MobileNetV3-Large": {
        "model_class": MobileNetV3_AudioClassifier,
        "args": {
            "num_classes": num_classes,
            "input_channels": 1,
            "pretrained": True  # Sử dụng pretrained weights từ ImageNet
        }
    }
}

Preparing dataset...
Number of classes: 8
Data split: 7316 train, 1830 test.


In [10]:
for model_name, config in model_configs.items():
    print(f"\n{'='*20} Evaluating: {model_name} {'='*20}")
    model_results = []
    for percent in DATA_PERCENTAGES:
        print(f"--- Training with {percent*100:.1f}% of labeled data ---")

        if percent < 1.0:
            stratify_labels = full_metadata['label'].iloc[train_indices].values
            subset_train_indices, _ = train_test_split(
                train_indices, train_size=percent, random_state=SEED, stratify=stratify_labels
            )
        else:
            subset_train_indices = train_indices

        train_metadata_subset = full_metadata.iloc[subset_train_indices]
        train_dataset = SpectrogramDataset(metadata_df=train_metadata_subset, data_dir=ROOT_DIR, transform=train_transform)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0, collate_fn=collate_fn_remove_none)

        checkpoint_filename = f"{model_name.replace(' ', '_')}_data_{percent*100:.1f}percent.pth"
        checkpoint_path = os.path.join(CHECKPOINT_DIR, checkpoint_filename)
        metrics_csv_path = os.path.join(CHECKPOINT_DIR, f"{model_name.replace(' ', '_')}_data_{percent*100:.1f}percent_metrics.csv")

        model = config["model_class"](**config["args"]).to(device)

        f1 = train_and_evaluate(model, train_loader, test_loader, epochs=100, checkpoint_path=checkpoint_path, metrics_csv_path=metrics_csv_path)
        model_results.append({'percent': percent, 'f1': f1})
        print(f"==> Results with {percent*100:.1f}% data: Best F1-Score = {f1:.4f}")
    results[model_name] = model_results

print("\n--- FINAL RESULTS ---")
for model_name, result_list in results.items():
    print(f"\nModel: {model_name}")
    for res in result_list:
        print(f"  {res['percent']*100:.1f}% data: F1-Score = {res['f1']:.4f}")


==================== Evaluating: MobileNetV3-Large ====================
--- Training with 100.0% of labeled data ---


Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-5c1a4163.pth" to C:\Users\PC/.cache\torch\hub\checkpoints\mobilenet_v3_large-5c1a4163.pth
100%|█████████████████████████████████████████████████████████████████████████████| 21.1M/21.1M [00:01<00:00, 11.8MB/s]
C:\Users\PC\anaconda3\envs\train_env\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/100:   0%|          | 0/229 [00:00<?, ?it/s]


Epoch 1: New best F1-score: 0.7057
Accuracy: 0.7301, Precision: 0.7409, Recall: 0.7301
Saving checkpoint to C:\Users\PC\lavar\audiotwin_checkpoints_medium\training_checkpoints\MobileNetV3-Large_data_100.0percent.pth


Epoch 2/100:   0%|          | 0/229 [00:00<?, ?it/s]

...

Epoch 97/100:   0%|          | 0/229 [00:00<?, ?it/s]

Epoch 98/100:   0%|          | 0/229 [00:00<?, ?it/s]

Epoch 99/100:   0%|          | 0/229 [00:00<?, ?it/s]

Epoch 100/100:   0%|          | 0/229 [00:00<?, ?it/s]

Finished Training for this run. Best F1-score: 0.9612
==> Results with 100.0% data: Best F1-Score = 0.9612

--- FINAL RESULTS ---

Model: MobileNetV3-Large
  100.0% data: F1-Score = 0.9612
